In [1]:
# # IMPORTS
# astronomy
from astropy.io import fits
# plotting 
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.pyplot as plt
import matplotlib as mpl
# data 
import numpy as np
# file IO
import csv
# my python files 
import match
import v2_AGN_DataAndPlotting as adp

In [2]:
def PrintNumber(myDict) :
    for key,val in myDict.items():
        print(key,len(val)) 

In [3]:
# read AGN IDs
agnIDs = adp.ReadFile(adp.path_csv+'\\v2_AGNid_tricolor.csv')

# convert agnIDs into dict of integer numpy arrays 
agnIDs_dict = {
    'RED' : np.array(agnIDs[0], dtype=int),
    'YEL' : np.array(agnIDs[1], dtype=int),
    'BLU' : np.array(agnIDs[2], dtype=int)
}

# verify nmber of sources
PrintNumber(agnIDs_dict)

Cropped:  ['Row 1 = MIR exclusive AGN with id, redshift, IRAC, and 24um (n=352). Row 2 = MIR-X-ray inclusive AGN with id, redshift, IRAC, and 24um (n=548). Row 3 = X-ray exclusive AGN with id, redshift, IRAC, and 24um (n=1227). ']
RED 352
YEL 548
BLU 1227


In [4]:
# get redshifts
zDict = adp.ReadRedshifts(file=adp.path_cat+'\COSMOS_z_matches.csv')
zDict_le = adp.GetRedshifts_lessEq(zMax=1,zDict=zDict)

Cropped:  ['ID', 'z']
Number of redshifts: 485793
Number of redshifts <=1: 211361
